In [7]:
import evaluate
import Utils
import network
import numpy as np

In [25]:
image_num = 180
model = network.Conv64Features()
D=2

In [26]:
# metoda vraća predikciju mape dispariteta
model.to('cpu').eval()
# dohvati slike
left = Utils.get_left_image(image_num)
right = Utils.get_right_image(image_num)
# pretvori slike za ulaz u model
left_model_input = evaluate.transform_image_for_model(left)
right_model_input = evaluate.transform_image_for_model(right)
# provuci slike kroz model i pretvori ih u prikladni ndarray
left_output = evaluate.transform_model_output_to_ndarray(model(left_model_input))
right_output = evaluate.transform_model_output_to_ndarray(model(right_model_input))

In [27]:
def similarity_at_d(left_output, right_output, d):
    # ndarray[HxWxC] -> ndarray[HxW]

    # metoda prima outpute modela lijeve i desne slike.
    # Vraća sličnost lijeve i desne slike. Desna slika je pomaknuta za disparitet d.
    shifted_right_output = np.roll(right_output, d, axis=1)
    return np.sum(left_output * shifted_right_output, axis=2)

In [28]:
left_output_custom = np.array([
    [[1,2],
     [2,2],
     [3,2]],
    [[4,3],
     [5,3],
     [6,3]]
])
right_output_custom = np.array([
    [[1,2],
     [2,2],
     [3,2]],
    [[4,3],
     [5,3],
     [6,3]]
])

In [29]:
similarity_at_0 = similarity_at_d(left_output_custom, right_output_custom, 0)

In [30]:
similarity_at_0

array([[ 5,  8, 13],
       [25, 34, 45]])

In [31]:
similarity_at_1 = similarity_at_d(left_output_custom, right_output_custom, 1)

In [32]:
similarity_at_1

array([[ 7,  6, 10],
       [33, 29, 39]])

In [33]:
# testiramo implementaciju predict_disp_map, D=2, što znači da će d biti 0 i 1

# ndarray[HxWxD]
similarities_at_all_D_disparities = np.stack([similarity_at_d(left_output_custom, right_output_custom, d) for d in range(D)], axis=2)

# ndarray[HxW]
predicted_disparity_map = np.argmin(similarities_at_all_D_disparities, axis=2)

In [34]:
similarities_at_all_D_disparities

array([[[ 5,  7],
        [ 8,  6],
        [13, 10]],

       [[25, 33],
        [34, 29],
        [45, 39]]])

In [35]:
predicted_disparity_map

array([[0, 1, 1],
       [0, 1, 1]], dtype=int64)